THƯ VIỆN

In [1]:
from keras.preprocessing import image
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten

---

KHAI BÁO

In [2]:
catergories = ['real', 'spoof']

In [7]:
need_dir = 'antispoofing_dataset'
train_dir = os.path.join(need_dir, 'train')
test_dir = os.path.join(need_dir, 'test')

In [19]:
link = './model_weights/model_16-0.890275.h5'

---

LOAD MODEL

In [14]:
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(160, 160, 3)))
mobilenet.trainable = False
output = Flatten()(mobilenet.output)
output = Dropout(0.3)(output)
output = Dense(units=8, activation='relu')(output)
prediction = Dense(1, activation='sigmoid')(output)
model = Model(inputs=mobilenet.input, outputs=prediction)

In [16]:
model.load_weights(link)

---

TESTING

In [17]:
def check_fake(path, catergory_type):
    predictor = {}
    path = os.path.join(path, catergory_type)
    for each_img in os.listdir(path):
        try:
            img = image.load_img(os.path.join(path, each_img), target_size=(160, 160))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = img / 255.0
            prediction = model.predict(img)
            if prediction > 0.5:
                prediction_class = 1
            else:
                prediction_class = 0
            result = catergories[prediction_class]
            if result not in predictor:
                predictor[result] = 1
            else:
                predictor[result] += 1
        except Exception as e:
            pass
    return predictor

In [ ]:
check_fake(test_dir, catergories[0])

In [ ]:
check_fake(test_dir, catergories[1])